# Solvers comparison

This notebook aims at providing a template to perform comparisons in computational time between different solvers.
To use this notebook, the user is expected to run the following procedure:
1. Setup the config.yaml file for the simulation
2. For every solver 'solver' under consideration:
   1. Setup the solver option in the config.yaml file 
   2. Run the workflow with snakemake
   3. Rename the folder benchmarks/solve_network into benchmarks/solve_network_{solver}
   4. Rename the folder results into results_{solver}
3. Modify the list 'solver_to_compare' below to specify the solvers to consider
4. Run the notebook

## Change the current directory to the main package folder

In [ ]:
# Specify the current directory
import sys

sys.path.append("../")  # to import helpers

from scripts._helpers import sets_path_to_root

sets_path_to_root("pypsa-africa")

## Specify the solvers to compare

In [ ]:
# list of solvers to compare
solvers_to_compare = ["gurobi", "highs"]

# list of configurations to compare and their description
configs_to_compare = {
    "elec_s_10_ec_lcopt_Co2L-1H": 10,  # "10-cluster",
    "elec_s_20_ec_lcopt_Co2L-1H": 20,  # "20-cluster",
    "elec_s_40_ec_lcopt_Co2L-1H": 40,  # "40-cluster",
    "elec_s_54_ec_lcopt_Co2L-1H": 54,  # "54-cluster",
}

## Import packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pypsa

## Compare computational resources
### Preload input data

In [ ]:
comp_time_by_solver = []

for solver in solvers_to_compare:
    for conf_name, conf_desc in configs_to_compare.items():
        tempdata = pd.read_csv(
            "benchmarks/solve_network_" + solver + "/" + conf_name, delimiter="\t"
        )
        comp_time_by_solver.append(
            [solver, conf_desc, conf_name] + list(tempdata.values[0])
        )

comp_time_by_solver = pd.DataFrame(
    comp_time_by_solver,
    columns=[
        "solver",
        "config",
        "filename",
        "s",
        "h:m:s",
        "max_rss",
        "max_vms",
        "max_uss",
        "max_pss",
        "io_in",
        "io_out",
        "mean_load",
        "cpu_time",
    ],
)

### Total computational time

In [ ]:
for solver in solvers_to_compare:
    plt.plot(
        configs_to_compare.values(),
        comp_time_by_solver.loc[comp_time_by_solver["solver"] == solver, "s"] / 3600,
        label=solver,
    )
plt.ylabel("Comp. time [h]")
# plt.yscale('log')
plt.grid()
plt.legend()
plt.xlabel("Number of clusters")

### Total workload

In [ ]:
for solver in solvers_to_compare:
    plt.plot(
        configs_to_compare.values(),
        comp_time_by_solver.loc[
            comp_time_by_solver["solver"] == solver, ["s", "mean_load"]
        ].product(axis=1)
        / 3600
        / 100,
        label=solver,
    )
plt.ylabel("Workload [h * avg load]")
# plt.yscale('log')
plt.grid()
plt.legend()
plt.xlabel("Number of clusters")

### Maximum virtual memory

In [ ]:
for solver in solvers_to_compare:
    plt.plot(
        configs_to_compare.values(),
        comp_time_by_solver.loc[comp_time_by_solver["solver"] == solver, "max_vms"]
        / 1000,
        label=solver,
    )
plt.ylabel("Memory [Gb]")
# plt.yscale('log')
plt.grid()
plt.legend()
plt.xlabel("Number of clusters")

## Comparison of the solutions
### Preload results

In [ ]:
comp_results_by_solver = []

for solver in solvers_to_compare:
    for conf_name, conf_desc in configs_to_compare.items():
        n = pypsa.Network("results_" + solver + "/networks/" + conf_name + ".nc")
        comp_results_by_solver.append(
            [solver, conf_desc, conf_name] + [n.objective, n.generators.p_nom_opt.sum()]
        )

comp_results_by_solver = pd.DataFrame(
    comp_results_by_solver,
    columns=["solver", "config", "filename", "objective", "p_nom_opt"],
)

### Plot of objective function

In [ ]:
for solver in solvers_to_compare:
    plt.plot(
        configs_to_compare.values(),
        comp_results_by_solver.loc[
            comp_results_by_solver["solver"] == solver, "objective"
        ]
        / 1000,
        label=solver,
    )
plt.ylabel("Memory [Gb]")
# plt.yscale('log')
plt.grid()
plt.legend()
plt.xlabel("Number of clusters")